# Extract Reece Account Balance

In [149]:
import smtplib
import time
import calendar
import smtplib
import os

#load config details 
file = open('../../../Teal_config/config.txt', 'r')

Filecontents=file.read()
Token=[Filecontents[6:26], Filecontents[34:40], Filecontents[50:71],Filecontents[83:103], Filecontents[111:200]]


# Get data from website
from six.moves.urllib.request import urlopen
link = Token[4]
response = urlopen(link)
content = response.read()
#Convert content to a string
str1 = content.decode("utf-8")

#Find index of RECENT in the webpage text
RecentInd=str1.find("RECENT")

#Find the next $ after recent
DollarInd=str1.find("$", RecentInd)
#Find the next < after $ (this marks the end of the wanted numbers)
triInd=str1.find("<", DollarInd)

#Find the balance
DollarBal=str1[DollarInd:triInd]

#Find the end of month balance
EOMInd=str1.find("Due End Of Month")
DollarEOMInd=str1.find("$", EOMInd)
triIndEOM=str1.find("<", DollarEOMInd)
DollarEOMBal=str1[DollarEOMInd:triIndEOM]

#Extract current date and time
date = time.strftime("%d/%m/%Y")
timenow = time.strftime("%X")
#Write Dollar Balance into a file
f= open("ReeceBal.txt","w+")
f.write("Reece Australia Pty Ltd, ABN 84 004 097 090\n")
f.write("Date and time = %s, %s\n\n" % (date, timenow))
f.write("Reece Bill due End of Month is %s\r\n" % DollarEOMBal)
f.write("Reece Account Balance since last bill %s\r\n" % DollarBal)
f.close() 

print ("Reece Bill due End of Month = %s" % DollarEOMBal)
print ("Reece Account Balance since last bill = %s" % DollarBal)

## Email details from gmail


## dd/mm/yyyy format
#print (time.strftime("%d/%m/%Y"))
date = time.strftime("%d/%m/%Y")
monthyr = time.strftime("%b %Y")


from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
#from email.MIMEMultipart import MIMEMultipart
#from email.MIMEText import MIMEText
 
try: 
    fromaddr = Token[0]
    toaddr = Token[2]  #tolist =["one@one.org","two@two.org","three@three.org","four@four.org"]
    ccaddr = Token[0]
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = toaddr
    msg['Cc'] = ccaddr
    msg['Subject'] = "Reece Account Bill"

    #body = "Reece Account Bill\n%s\nAmount Due since last email %s" % (date, DollarBal)
    body = """REECE ACCOUNT BILL \n
            Reece Australia Pty Ltd, ABN 84 004 097 090\n\n
            Due End of Month (due end %s) = %s\n 
            Reece Account Balance since last bill (up to %s) = %s""" % (monthyr,DollarEOMBal,date,DollarBal) 
    msg.attach(MIMEText(body, 'plain'))

    #attach file to email
    filename = "ReeceBal.txt"
    attachment = open("ReeceBal.txt", "r")

    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    msg.attach(part)

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(fromaddr, Token[1])
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    server.quit()
except:  
    print('Something went wrong...')



Reece Bill due End of Month = $1,686.69
Reece Account Balance since last bill = $1,273.02
